In [ ]:
!pip install pandarallel
import pandas as pd
from bs4 import BeautifulSoup
import requests
import glob
import numpy as np
from pandarallel import pandarallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandarallel: filename=pandarallel-1.6.2-py3-none-any.whl size=16411 sha256=3b2af71c6ec021df3fee968ccbd9143563dcffaab37ac8f8811536ab299adcb7
  Stored in directory: /root/.cache/pip/wheels/01/8d/99/4ce6a1e914ed6ec383bc099dfbe0112dfde0bc39957ca93ec1
Successfully built pandarallel


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.DataFrame()
read_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/noticiasdegipuzkoa/url.csv', header=[0])
df=pd.concat([df,read_file]).reset_index(drop=True)
print(len(df))

115694


In [ ]:
def get_articles(url):
  try:
    r1 = requests.get(url)
    coverpage = r1.content
    soup = BeautifulSoup(coverpage, 'html.parser')
    
    try:
      coverpage=soup.find_all('div',class_='headline-article')
      title=coverpage[0].find('h1').get_text()
      try:
       description=coverpage[0].find('h2').get_text()
      except:
        description=''
    except:
      title=''
      description=''
    
    try:
      coverpage=soup.find_all('span',class_='article-author__date')
      time=coverpage[0].get_text()
    except:
      time=''  
    article=''
    try:
      coverpage=soup.find_all('div',class_='article-body ')
      for i in coverpage[0].find_all('p'):
        article=article+i.get_text()
      
      # print(article)
    except:
      print('article error\n'+url)


  except:
    title=''
    description=''
    time=''
    article=''
    print('url error\n'+url)    

  return [title,description,time,article]

In [ ]:
get_articles('https://www.noticiasdegipuzkoa.eus/politica/2022/08/06/cuelgan-pancarta-monarquia-cerca-palacio-5890050.html')

['Cuelgan una pancarta contra la monarquía cerca del Palacio de Marivent',
 'Militantes de Frente Obrero ya realizaron una acción similar el año pasado',
 '\n                    06·08·22\n            ',
 '\n        Militantes de Frente Obrero han colgado este sábado una pancarta contra la monarquía frente a la Playa de Cala Mayor, en la Avenida Joan Miró, a escasos minutos del Palacio de Marivent.\n\n        Según han explicado en nota de prensa, de este modo quieren mostrar, "un año más", su "rechazo" a la monarquía. Cabe recordar que en agosto del año pasado realizaron una acción similar al colgar una pancarta de más de diez metros frente a La Seu de Palma.\n\n        Por otro lado, aprovechando una reciente visita guiada, militantes de la organización se "colaron" en el balcón del Ayuntamiento de Palma y colgaron una pancarta censurando al regidor Alberto Jarabo por prohibir las recogidas de alimentos en las inmediaciones de los supermercados, "dificultando la actividad de aquellos 

In [7]:
pandarallel.initialize(nb_workers= 30, progress_bar=True)

output=df['rows'].parallel_apply(get_articles)

df['country']='Guipúzcoa'
df['title']=''
df['description']=''
df['time']=''
df['article']=''

for i in range(0,len(df)):
  df.loc[i]['title','description','time','article']=output[i]

df = df[df.title.str.len()!=0].reset_index(drop=True)
df = df.dropna()
str_name='/content/drive/MyDrive/Colab Notebooks/noticiasdegipuzkoa/url_article.csv'
print(df)
df.to_csv(str_name, index=False)

INFO: Pandarallel will run on 30 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


article error
https://www.noticiasdegipuzkoa.eus/tags/union-europea/
article error
https://www.noticiasdegipuzkoa.eus/tags/politica/
article error
https://www.noticiasdegipuzkoa.eus/tags/mundo/
article error
https://www.noticiasdegipuzkoa.eus/tags/fuerzas-politicas/
article error
https://www.noticiasdegipuzkoa.eus/tags/politica-territorial/
article error
https://www.noticiasdegipuzkoa.eus/tags/politica-fiscal/
article error
https://www.noticiasdegipuzkoa.eus/tags/vuelta-al-mundo/
article error
https://www.noticiasdegipuzkoa.eus/tags/politica-penitenciaria/
article error
https://www.noticiasdegipuzkoa.eus/tags/politica-linguistica/
article error
https://www.noticiasdegipuzkoa.eus/tags/politica-catalana/
article error
https://www.noticiasdegipuzkoa.eus/tags/copa-del-mundo/
article error
https://www.noticiasdegipuzkoa.eus/actualidad/mundo/
article error
https://www.noticiasdegipuzkoa.eus/tendencias/que-mundo/
article error
https://www.noticiasdegipuzkoa.eus/actualidad/politica/
article er